In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import win32com.client as win32
from pathlib import Path
import os
import xlwings as xw
from datetime import date, timedelta


Read xls file then convert it into xlsx file

In [2]:
def convert_xls(input_path, output_path):
    try:
        app = xw.App(visible=False)
        wb = app.books.open(input_path)
        
        # Save the file as xlsx
        wb.save(output_path)

        # Close the workbook and quit Excel
        wb.close()
        app.quit()
        return True
    except:
        return False

In [6]:
import xlwings as xw

# Read xls file then convert it into xlsx file
def convert_xls(input_path, output_path):
    try:
        try:
            app = xw.App(visible=False)
            wb = app.books.open(input_path)
        except:
            print('Cannot open file')
        # Save the file as xlsx
        wb.save(output_path)

        # Close the workbook and quit Excel
        wb.close()
        app.quit()
        return True
    except:
        return False



c:\Users\Admin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Đọc file và tiền xử lý, sau đó lưu vào df

In [3]:
# Rewrite the header
""""
This function is used to transfrom the original dataframe with 
inapproriate columns name and structure.

Input: raw df read from excel files. See excel files to explore the structure
Output: df with columns that has been renamed, and

"""
def clean_df(df):
    col_rename = df.loc[0].to_list()[:8]
    for i, col in enumerate(df.columns):
        if i == 8:
            break
        df.rename(columns={col: col_rename[i]},inplace=True)
    df = df.drop(index=0)
    df_melt = pd.melt(df, id_vars=df.columns[0:8], value_vars=df.columns[8:],
                  var_name = 'Store', value_name='Order_Qty')
    return df_melt


Số SKU mỗi store đặt

In [4]:
# df_melt.dropna(subset='Order_Qty')
def store_group(df):
    nSKU_by_Store = df.groupby(by='Store')['Order_Qty'].count().sort_values(ascending=False).reset_index()
    stores_ordered_today = nSKU_by_Store[nSKU_by_Store['Order_Qty']>0]
    stores_has_no_order = nSKU_by_Store[nSKU_by_Store['Order_Qty']==0]
    return stores_ordered_today, stores_has_no_order

Function to get order list of an input store

In [5]:
def order_list(store,df):
    return df[(df['Store'] == store) &
                   (~df['Order_Qty'].isna())]

Function of exporting a df to csv file

In [6]:
def export_df(df, filename):
    # today = datetime.now().strftime(format='%d-%m')
    df.to_excel('output_files/' + filename + '.xlsx', index=False)

In [7]:
def main(order_file_path = '', order_store = False, no_order_store = False, store_list = []):
    date_string = (date.today() + timedelta(1)).strftime(format='%d_%m_%Y')
    if order_file_path == '':
        xls_filename = 'DFP_FreshFood_Order_' + date_string + '.xls'
        # Get the path to today's downloaded xls file
        xls_file_path = os.getcwd() + '\\daily_xls\\' + xls_filename
        # Get the path to the intended output with xlsx extension:
        xlsx_file_path = Path(os.getcwd() + '\\daily_xlsx\\' + xls_filename).with_suffix(".xlsx")
    else:
        xls_file_path = order_file_path
        xlsx_file_path = str(xls_file_path).replace('xls','xlsx')

    if not convert_xls(input_path=xls_file_path, output_path=xlsx_file_path):
        print(xlsx_file_path)
        print(xls_file_path)
        print('Fail to convert xls file')
        return
    else:
        df = clean_df(pd.read_excel(xlsx_file_path))
        # date = str(xlsx_file_path)[-15:-5]
        if order_store:
            store_with_orders, store_without_orders = store_group(df)
            export_df(store_with_orders, date_string + '_store_có_đặt')
        if no_order_store:
            store_with_orders, store_without_orders = store_group(df)
            export_df(store_without_orders, date_string +'_store_không_đặt')
        if store_list.__len__() != 0:
            for store in store_list:
                order_by_store = order_list(store, df)
                export_df(order_by_store, date_string + '_' + str(store))

In [8]:

main(no_order_store=True, order_store=True)

In [9]:
# D:\SGC\TPTS_Project\df_portal_order_check\daily_xls\DFP_FreshFood_Order_05_04_2024.xls